In [1]:
import gpt_2_simple as gpt2
from big_phoney import BigPhoney
import numpy as np

# model_name = "124M"
# model is saved into current directory under /models/124M/
# gpt2.download_gpt2(model_name=model_name) #need to run only once. comment out once done.

# Run this once per session? 
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')


/Users/stasianik/opt/anaconda3/envs/insight/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/stasianik/opt/anaconda3/envs/insight/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/stasianik/opt/anaconda3/envs/insight/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/stasianik/op

Loading checkpoint checkpoint/run1/model-500
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-500


In [2]:
# Later on I'll need to add the artist as a parameter, if I have 3, so it can load in the right finetuned model
def generate_lyrics(): 
    
    generated_text = gpt2.generate(sess, 
                     length=250,
                     temperature=0.9,
                     prefix="This is a placeholder for user input",
                     nsamples=1,
                     batch_size=1,
                     return_as_list=True)[0]
    
    target_lyrics = ["The snow glows white on the mountain tonight",
    "Not a footprint to be seen",
    "A kingdom of isolation",
    "And it looks like I'm the queen",
    "The wind is howling like this swirling storm inside",
    "Couldn't keep it in, heaven knows I've tried",
    "Don't let them in, don't let them see",
    "Be the good girl you always have to be",
    "Conceal, don't feel, don't let them know",
    "Well, now they know",
    "Let it go, let it go",
    "Can't hold it back anymore",
    "Let it go, let it go",
    "Turn away and slam the door",
    "I don't care what they're going to say",
    "Let the storm rage on",
    "The cold never bothered me anyway"]
    
    return generated_text, target_lyrics

In [10]:
lyrics = generate_lyrics()
gen_lyrics = lyrics[0]
print(gen_lyrics)

This is a placeholder for user input
- pose the question - no, no, no, no, no
- hey, hey, hey, hey
welcome to utopian-land

where all is new
where all is old
where all is utopian
where all is run aground
where you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are...

ive been dreaming of this place
ive been dreaming of it all
you know its sad, but its true
there is happiness in the sight
and sound
and that which we hear
and that which we do
we have reached the end of our wagon
somewhere else
else
wheres our sorrow?
wheels roll on the floor
its time to get up
and shake hands
and tell our heroes
who you are
theyre the ones who knew us well
and we are the wretched slaves
we are the wretched serfs
we are the wretched executioners
we are the wretched gendarmes running screaming
oh yeah
the crying, the crying, the crying
msgs going live the way you always planned
out in the warmth



In [14]:
# One step at a time
gen_lyrics[1]

'- pose the question - no, no, no, no, no'

In [15]:
a = gen_lyrics[1]
phoney = BigPhoney()
phoney.count_syllables(a)

9

In [18]:
schema = []
for line in gen_lyrics:
    syls = phoney.count_syllables(line)
    schema.append(syls)

11
9
4
8
0
4
4
7
6
31
0
7
7
7
8
2
5
5
10
3
1
5
5
5
3
6
3
7
7
6
10
11
2
9
11
4


In [19]:
print(schema)

[11, 9, 4, 8, 0, 4, 4, 7, 6, 31, 0, 7, 7, 7, 8, 2, 5, 5, 10, 3, 1, 5, 5, 5, 3, 6, 3, 7, 7, 6, 10, 11, 2, 9, 11, 4]


In [33]:
def count_syllables_generated(generated_text):
    
    phoney = BigPhoney()
    schema = []
    
    for line in generated_text:
        syls = phoney.count_syllables(line)
        schema.append(syls)
        
    return schema

In [ ]:
count_syllables_generated(gen_lyrics)

In [22]:
print(gen_lyrics)

['This is a placeholder for user input', '- pose the question - no, no, no, no, no', '- hey, hey, hey, hey', 'welcome to utopian-land', '', 'where all is new', 'where all is old', 'where all is utopian', 'where all is run aground', 'where you are you are you are you are you are you are you are you are you are you are you are you are you are you are you are...', '', 'ive been dreaming of this place', 'ive been dreaming of it all', 'you know its sad, but its true', 'there is happiness in the sight', 'and sound', 'and that which we hear', 'and that which we do', 'we have reached the end of our wagon', 'somewhere else', 'else', 'wheres our sorrow?', 'wheels roll on the floor', 'its time to get up', 'and shake hands', 'and tell our heroes', 'who you are', 'theyre the ones who knew us well', 'and we are the wretched slaves', 'we are the wretched serfs', 'we are the wretched executioners', 'we are the wretched gendarmes running screaming', 'oh yeah', 'the crying, the crying, the crying', 'msg

In [ ]:
#
text = generated_text[0]
text = text.splitlines()

In [ ]:
phoney = BigPhoney()

In [ ]:
a = text[1]

In [ ]:
phoney.count_syllables(a)

In [ ]:
def count_syllables_target(target_lyrics):

return target_schema

In [ ]:
def fit_lyrics(generated_text, target_schema)


return processed_lyrics

In [ ]:
print(generated_text[0])

In [ ]:
count_syllables(target_lyrics,generated_text)

In [ ]:
fit_lyrics(generated_text, target_schema)

In [ ]:
text = generated_text[0]

In [ ]:
print(text)